In [4]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from datetime import time
import os



In [5]:
in_file="raw_data_28-April'25.csv"
out_file="all_flow_data_27th April'25.csv"

In [6]:

raw_data=pd.read_csv(in_file,low_memory=False,memory_map=True)
data=pd.read_csv("format.csv",memory_map=True)
table_data=pd.read_csv("table_data.csv",memory_map=True)    



In [7]:
raw_data.head(3)

,cust_id,lead_id,fse_cust_id,workflow_node_id,sub_stage,created_at,updated_at,solution_type_level_2,solution_type_level_3,entity_type,products_context,sbstage_count,name_match_status,mid,mid_creation_date,agent_team,qc_pending_max_created,panelqc_max_created,final_timestamp,sub_stage1
0,1.335345e+09,3ca188f5-7e3f-481c-9bd5-d5070f57332d,1364330104,112,LEAD_CREATED,2025-04-02 15:04:42.000,2025-04-02 15:04:42.000,NaN,NaN,INDIVIDUAL,NaN,NaN,NaN,NaN,NaN,General Trade QR_SB_EDC,NaN,NaN,2025-04-02 15:04:42.000,NaN
1,1.460420e+09,f45b4500-5d21-4d41-b312-a38927a154c0,1366146851,112,LEAD_CREATED,2025-04-07 14:11:43.000,2025-04-07 14:11:43.000,NaN,NaN,INDIVIDUAL,NaN,NaN,False,NaN,NaN,General Trade QR_SB_EDC,NaN,NaN,2025-04-07 14:11:43.000,NaN
2,1.446959e+09,ac18a7c8-0266-4bd1-a4f5-a3b79e2e1879,1120140098,112,LEAD_CREATED,2025-04-26 12:00:13.000,2025-04-26 12:00:13.000,NaN,NaN,INDIVIDUAL,"[""Paytm QR""]",NaN,NaN,NaN,NaN,General Trade QR_SB_EDC,NaN,NaN,2025-04-26 12:00:13.000,NaN


In [8]:

raw_data.insert(5,'q',0)
len(raw_data)

546998

In [9]:
table_map = table_data.set_index('NODE ID')['Lead Status'].to_dict()
raw_data['q'] = raw_data['workflow_node_id'].map(table_map)



In [10]:
raw_data.loc[raw_data['sub_stage1']=='QC_AUTO_APPROVED','q']='Leads QC Auto Approved'

In [11]:
raw_data['q'].value_counts()

Merchant Live                                                       318059
Lead Created/Posted (Pending at Sales team for  Document Upload)    174528
Documents Rejected (Pending at Sales Team)                           29095
Lead stuck in Intermediate Stages (Dedupe/Risk/PG rejections)         2914
Leads QC Auto Approved                                                  20
Leads Rejected in Risk Review                                           19
Name: q, dtype: int64

In [12]:
raw_data=raw_data.dropna(subset=['q'])

raw_data['q'].value_counts()

Merchant Live                                                       318059
Lead Created/Posted (Pending at Sales team for  Document Upload)    174528
Documents Rejected (Pending at Sales Team)                           29095
Lead stuck in Intermediate Stages (Dedupe/Risk/PG rejections)         2914
Leads QC Auto Approved                                                  20
Leads Rejected in Risk Review                                           19
Name: q, dtype: int64

In [13]:
raw_data['remove_date']=raw_data['updated_at'].copy()

raw_data['remove_date'] = pd.to_datetime(raw_data['remove_date'])

raw_data['remove_date'] = raw_data['remove_date'].dt.strftime('%Y-%m-%d')

raw_data['remove_date'] = pd.to_datetime(raw_data['remove_date'])
len(raw_data)


524635

In [14]:
#print(pd.Timestamp.now().normalize()-timedelta(days=1))

In [15]:

current_timestamp = pd.Timestamp.now().normalize()

raw_data=raw_data[raw_data['remove_date']!=current_timestamp]

In [16]:
raw_data=raw_data.drop(columns=['remove_date'])



In [17]:
raw_data['solution_type_level_3'].unique()

array([nan, 'ACCOUNT_UPGRADE_LENDING', 'ACCOUNT_UPGRADE', 'KYC_UPDATE',
       'PROFILE_UPDATE', 'DIY', 'YBL_LIMIT_ENHANCEMENT'], dtype=object)

In [18]:
raw_data['solution_type_level_3'].value_counts()

KYC_UPDATE                 28249
ACCOUNT_UPGRADE_LENDING    10152
ACCOUNT_UPGRADE             1581
PROFILE_UPDATE               166
DIY                           33
YBL_LIMIT_ENHANCEMENT          1
Name: solution_type_level_3, dtype: int64

In [19]:
raw_data=raw_data[raw_data['solution_type_level_3']!='YBL_LIMIT_ENHANCEMENT']

raw_data['solution_type_level_3'].unique()

array([nan, 'ACCOUNT_UPGRADE_LENDING', 'ACCOUNT_UPGRADE', 'KYC_UPDATE',
       'PROFILE_UPDATE', 'DIY'], dtype=object)

In [20]:
raw_data['solution_type_level_3']=raw_data['solution_type_level_3'].replace(['ACCOUNT_UPGRADE_LENDING','PROFILE_UPDATE'],'ACCOUNT_UPGRADE')

raw_data['solution_type_level_3'].unique()

array([nan, 'ACCOUNT_UPGRADE', 'KYC_UPDATE', 'DIY'], dtype=object)

In [21]:
raw_data.loc[raw_data['agent_team']=='3P FSE','solution_type_level_3']='3P FSE'

In [22]:
raw_data['solution_type_level_3'].value_counts()

KYC_UPDATE         27853
ACCOUNT_UPGRADE    11899
3P FSE               642
DIY                   33
Name: solution_type_level_3, dtype: int64

In [23]:
print(len(raw_data))
raw_data.head(3)

517290


,cust_id,lead_id,fse_cust_id,workflow_node_id,sub_stage,q,created_at,updated_at,solution_type_level_2,solution_type_level_3,...,products_context,sbstage_count,name_match_status,mid,mid_creation_date,agent_team,qc_pending_max_created,panelqc_max_created,final_timestamp,sub_stage1
0,1.335345e+09,3ca188f5-7e3f-481c-9bd5-d5070f57332d,1364330104,112,LEAD_CREATED,Lead Created/Posted (Pending at Sales team for...,2025-04-02 15:04:42.000,2025-04-02 15:04:42.000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,General Trade QR_SB_EDC,NaN,NaN,2025-04-02 15:04:42.000,NaN
1,1.460420e+09,f45b4500-5d21-4d41-b312-a38927a154c0,1366146851,112,LEAD_CREATED,Lead Created/Posted (Pending at Sales team for...,2025-04-07 14:11:43.000,2025-04-07 14:11:43.000,NaN,NaN,...,NaN,NaN,False,NaN,NaN,General Trade QR_SB_EDC,NaN,NaN,2025-04-07 14:11:43.000,NaN
2,1.446959e+09,ac18a7c8-0266-4bd1-a4f5-a3b79e2e1879,1120140098,112,LEAD_CREATED,Lead Created/Posted (Pending at Sales team for...,2025-04-26 12:00:13.000,2025-04-26 12:00:13.000,NaN,NaN,...,"[""Paytm QR""]",NaN,NaN,NaN,NaN,General Trade QR_SB_EDC,NaN,NaN,2025-04-26 12:00:13.000,NaN


In [24]:
data[['Merchant Cust ID', 'Lead_ID', 'Entity_Type', 'Node id', 'Lead_Stage',
       'Lead Status', 'Created Timestamp', 'Updated Timestamp',
       'Product_Context', 'Name_Match_Status',
       'Final TimeStamp', 'QC Panel Successful TimeStamp',
       'Last QC Pending Timestamp',
       'Stage movement count', 'cust_id', 'agent_team',
       'mid', 'mid_created_date', 'solution_type_level_3']]=raw_data[['cust_id', 'lead_id','entity_type','workflow_node_id', 'sub_stage',
       'q', 'created_at', 'updated_at','products_context','name_match_status','final_timestamp','panelqc_max_created','qc_pending_max_created','sbstage_count','fse_cust_id','agent_team','mid','mid_creation_date','solution_type_level_3']]
       
len(data)

517290

In [25]:
data.head(3)


,Merchant Cust ID,Lead_ID,Entity_Type,Node id,Lead_Stage,Lead Status,Created Timestamp,Updated Timestamp,Ageing,Product_Context,...,Working TimeSlab,New Off Time,New Last QC Pending Time,Stage movement count,Rejection Ageing,cust_id,agent_team,mid,mid_created_date,solution_type_level_3
0,1.335345e+09,3ca188f5-7e3f-481c-9bd5-d5070f57332d,INDIVIDUAL,112,LEAD_CREATED,Lead Created/Posted (Pending at Sales team for...,2025-04-02 15:04:42.000,2025-04-02 15:04:42.000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1364330104,General Trade QR_SB_EDC,NaN,NaN,NaN
1,1.460420e+09,f45b4500-5d21-4d41-b312-a38927a154c0,INDIVIDUAL,112,LEAD_CREATED,Lead Created/Posted (Pending at Sales team for...,2025-04-07 14:11:43.000,2025-04-07 14:11:43.000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1366146851,General Trade QR_SB_EDC,NaN,NaN,NaN
2,1.446959e+09,ac18a7c8-0266-4bd1-a4f5-a3b79e2e1879,INDIVIDUAL,112,LEAD_CREATED,Lead Created/Posted (Pending at Sales team for...,2025-04-26 12:00:13.000,2025-04-26 12:00:13.000,NaN,"[""Paytm QR""]",...,NaN,NaN,NaN,NaN,NaN,1120140098,General Trade QR_SB_EDC,NaN,NaN,NaN


In [26]:
data['Updated Timestamp']=pd.to_datetime(data['Updated Timestamp'])
data['Created Timestamp']=pd.to_datetime(data['Created Timestamp'])
data['QC Panel Successful TimeStamp']=pd.to_datetime(data['QC Panel Successful TimeStamp'])


In [27]:
#current_date = datetime.now() - timedelta(days=2)

#current_timestamp = pd.Timestamp.now().normalize()-timedelta(days=1)


#print(current_timestamp)

#print(current_date)

In [28]:
user_date = '2024-10-15'  # Example: YYYY-MM-DD format

# Convert the combined string into a datetime object
custom_timestamp = datetime.strptime(user_date, '%Y-%m-%d')

print(custom_timestamp)
print(type(custom_timestamp))

current_day = custom_timestamp.day
current_month = custom_timestamp.month
current_year = custom_timestamp.year

print(current_day)
print(current_month)
print(current_year)


2024-10-15 00:00:00
<class 'datetime.datetime'>
15
10
2024


In [29]:
current_date = datetime.now() - timedelta(days=7)
current_day = current_date.day
current_month = current_date.month
current_year = current_date.year
print(current_date)

data.loc[(data['Updated Timestamp'].dt.day>=current_day) &
         (data['Updated Timestamp'].dt.month==current_month) &
         (data['Updated Timestamp'].dt.year==current_year), 'Ageing']='Last 7 days'

2025-04-21 13:17:16.044644


In [30]:
current_date = datetime.now() - timedelta(days=1)
current_day = current_date.day
current_month = current_date.month
current_year = current_date.year
current_date

data.loc[(data['Updated Timestamp'].dt.day==current_day) &
         (data['Updated Timestamp'].dt.month==current_month) &
         (data['Updated Timestamp'].dt.year==current_year), 'Ageing']='Yesterday'

In [31]:
data['Ageing']=data['Ageing'].fillna('')

In [32]:
data['Ageing'].value_counts()

               346900
Last 7 days    167628
Yesterday        2762
Name: Ageing, dtype: int64

In [33]:


data.loc[data['Name_Match_Status']==True, ['Name_Match_Status', '10K/1rs MID Status']]=['Name Matched','10k MID created']

data.loc[data['Name_Match_Status']==False, ['Name_Match_Status', '10K/1rs MID Status']]=['Name Not Matched','1Rs MID created']

In [34]:
data['Name_Match_Status']=data['Name_Match_Status'].fillna('')

In [35]:
data['Name_Match_Status'].value_counts()

Name Matched        320651
                    115403
Name Not Matched     81236
Name: Name_Match_Status, dtype: int64

In [36]:
data['Product_Context'].value_counts()

["Paytm QR"]     289105
["EDC"]           34543
["TAP_N_PAY"]      1198
[]                   17
Name: Product_Context, dtype: int64

In [37]:
data.loc[data['Product_Context'].isin(['[]','["EDC"]','["TAP_N_PAY"]']),'Product_Context']='EDC'
data['Product_Context'].value_counts()

["Paytm QR"]    289105
EDC              35758
Name: Product_Context, dtype: int64

In [38]:
data.loc[~data['Product_Context'].isin(['EDC']),'Product_Context']='NON_EDC'
data['Product_Context'].value_counts()

NON_EDC    481532
EDC         35758
Name: Product_Context, dtype: int64

In [39]:
data['mid_created_date']=pd.to_datetime(data['mid_created_date'],errors='coerce')

In [40]:
data['mid_created_date'].head(10)

0                   NaT
1                   NaT
2                   NaT
3   2025-04-10 19:44:51
4   2025-04-16 14:17:26
5   2025-04-09 17:58:08
6   2025-04-05 15:00:40
7   2025-04-19 11:58:05
8   2019-03-02 19:18:14
9   2025-04-11 11:35:06
Name: mid_created_date, dtype: datetime64[ns]

In [41]:
data.columns

Index(['Merchant Cust ID', 'Lead_ID', 'Entity_Type', 'Node id', 'Lead_Stage',
       'Lead Status', 'Created Timestamp', 'Updated Timestamp', 'Ageing',
       'Product_Context', 'Name_Match_Status', '10K/1rs MID Status',
       'Final TimeStamp', 'QC Panel Successful TimeStamp',
       'Last QC Pending Timestamp', 'TAT', 'Merchant QC TAT slab',
       'Working TimeSlab', 'New Off Time', 'New Last QC Pending Time',
       'Stage movement count', 'Rejection Ageing', 'cust_id', 'agent_team',
       'mid', 'mid_created_date', 'solution_type_level_3'],
      dtype='object')

In [42]:
current_date = datetime.now() - timedelta(days=7)
current_day = current_date.day
current_month = current_date.month
current_year = current_date.year
print(current_date)

data.loc[ (data['Lead Status']== 'Documents Rejected (Pending at Sales Team)') & ((data['Updated Timestamp'].dt.day<current_day) &
         (data['Updated Timestamp'].dt.month==current_month) &
         (data['Updated Timestamp'].dt.year==current_year)), 'Rejection Ageing']='More Than 7 Days'

2025-04-21 13:17:18.638032


In [43]:
current_date = datetime.now() - timedelta(days=7)
current_day = current_date.day
current_month = current_date.month
current_year = current_date.year
current_date

data.loc[ (data['Lead Status']== 'Documents Rejected (Pending at Sales Team)') & ((data['Updated Timestamp'].dt.day>=current_day) &
         (data['Updated Timestamp'].dt.month==current_month) &
         (data['Updated Timestamp'].dt.year==current_year)), 'Rejection Ageing']='6-7 Days'

In [44]:
current_date = datetime.now() - timedelta(days=5)
current_day = current_date.day
current_month = current_date.month
current_year = current_date.year
current_date

data.loc[ (data['Lead Status']== 'Documents Rejected (Pending at Sales Team)') & ((data['Updated Timestamp'].dt.day>=current_day) &
         (data['Updated Timestamp'].dt.month==current_month) &
         (data['Updated Timestamp'].dt.year==current_year)), 'Rejection Ageing']='4-5 Days'

In [45]:

current_date = datetime.now() - timedelta(days=3)
current_day = current_date.day
current_month = current_date.month
current_year = current_date.year
current_date

data.loc[ (data['Lead Status']== 'Documents Rejected (Pending at Sales Team)') & ((data['Updated Timestamp'].dt.day>=current_day) &
         (data['Updated Timestamp'].dt.month==current_month) &
         (data['Updated Timestamp'].dt.year==current_year)), 'Rejection Ageing']='2-3 Days'

In [46]:
current_date = datetime.now() - timedelta(days=1)
current_day = current_date.day
current_month = current_date.month
current_year = current_date.year
current_date

data.loc[ (data['Lead Status']== 'Documents Rejected (Pending at Sales Team)') & ((data['Updated Timestamp'].dt.day==current_day) &
         (data['Updated Timestamp'].dt.month==current_month) &
         (data['Updated Timestamp'].dt.year==current_year)), 'Rejection Ageing']='Less Than 1 Day'

In [47]:
data['Rejection Ageing'].value_counts()

More Than 7 Days    18992
2-3 Days             3523
4-5 Days             3239
6-7 Days             2667
Less Than 1 Day       219
Name: Rejection Ageing, dtype: int64

In [48]:
data['Rejection Ageing'].value_counts()

More Than 7 Days    18992
2-3 Days             3523
4-5 Days             3239
6-7 Days             2667
Less Than 1 Day       219
Name: Rejection Ageing, dtype: int64

In [49]:
data.loc[~data['Entity_Type'].isin(['INDIVIDUAL','PROPRIETORSHIP']),'Entity_Type']='NON_PROPRIETORSHIP'

In [50]:
data['Entity_Type'].value_counts()

INDIVIDUAL            469467
PROPRIETORSHIP         44059
NON_PROPRIETORSHIP      3764
Name: Entity_Type, dtype: int64

In [51]:
data.columns

Index(['Merchant Cust ID', 'Lead_ID', 'Entity_Type', 'Node id', 'Lead_Stage',
       'Lead Status', 'Created Timestamp', 'Updated Timestamp', 'Ageing',
       'Product_Context', 'Name_Match_Status', '10K/1rs MID Status',
       'Final TimeStamp', 'QC Panel Successful TimeStamp',
       'Last QC Pending Timestamp', 'TAT', 'Merchant QC TAT slab',
       'Working TimeSlab', 'New Off Time', 'New Last QC Pending Time',
       'Stage movement count', 'Rejection Ageing', 'cust_id', 'agent_team',
       'mid', 'mid_created_date', 'solution_type_level_3'],
      dtype='object')

In [52]:
data['Lead Status'].value_counts()

Merchant Live                                                       314510
Lead Created/Posted (Pending at Sales team for  Document Upload)    171355
Documents Rejected (Pending at Sales Team)                           28640
Lead stuck in Intermediate Stages (Dedupe/Risk/PG rejections)         2747
Leads QC Auto Approved                                                  20
Leads Rejected in Risk Review                                           18
Name: Lead Status, dtype: int64

In [53]:
null_rejected_timeStamp=data[(data['Lead Status']=='Documents Rejected (Pending at Sales Team)')&(data['Last QC Pending Timestamp'].isnull())]


null_rejected_timeStamp=null_rejected_timeStamp[['Lead_ID']]

print(len(null_rejected_timeStamp))

null_rejected_timeStamp['id']="'" + null_rejected_timeStamp['Lead_ID'] + "'" + ","

null_rejected_timeStamp.head(3)

null_rejected_timeStamp.to_csv("rejected_null.csv", index=False)


0


In [54]:
#dghdsg

In [55]:
# raise Exception("Stop execution here")

print(len(data))

517290


In [56]:
rejected_null_value=pd.read_csv('rejected_null_value.csv')

rejected_null_value['q']=pd.to_datetime(rejected_null_value['q'],format='%d-%m-%Y %H:%M:%S')

print(rejected_null_value.columns)

Index(['lead_id', 'q'], dtype='object')


In [57]:
print(rejected_null_value.head(3))
print(len(data))

Empty DataFrame
Columns: [lead_id, q]
Index: []
517290


In [58]:

# Aggregate the right DataFrame by taking the first occurrence per lead_id
#rejected_agg = rejected_null_value.groupby('lead_id').first().reset_index()

# Merge the DataFrames on the single column
#data = data.merge(rejected_agg[['lead_id', 'Status']], left_on='Lead_ID', right_on='lead_id', how='left')

# Fill in values based on conditions
#data.loc[(data['Lead Status'] == 'Documents Rejected (Pending at Sales Team)') & (data['Last QC Pending Timestamp'].isnull()), 'Last QC Pending Timestamp'] = data['q']


In [59]:
data=data.merge(rejected_null_value,left_on='Lead_ID',right_on='lead_id',how='left')

data.loc[(data['Lead Status']=='Documents Rejected (Pending at Sales Team)') & (data['Last QC Pending Timestamp'].isnull()) ,'Last QC Pending Timestamp']=data['q']

data=data.drop(columns=['lead_id','q'])

print(data.columns)
print(len(data[(data['Lead Status']=='Documents Rejected (Pending at Sales Team)')&(data['Last QC Pending Timestamp'].isnull())]))
print(len(data))

Index(['Merchant Cust ID', 'Lead_ID', 'Entity_Type', 'Node id', 'Lead_Stage',
       'Lead Status', 'Created Timestamp', 'Updated Timestamp', 'Ageing',
       'Product_Context', 'Name_Match_Status', '10K/1rs MID Status',
       'Final TimeStamp', 'QC Panel Successful TimeStamp',
       'Last QC Pending Timestamp', 'TAT', 'Merchant QC TAT slab',
       'Working TimeSlab', 'New Off Time', 'New Last QC Pending Time',
       'Stage movement count', 'Rejection Ageing', 'cust_id', 'agent_team',
       'mid', 'mid_created_date', 'solution_type_level_3'],
      dtype='object')
0
517290


In [60]:
data['Final TimeStamp']=pd.to_datetime(data['Final TimeStamp'])
data['Last QC Pending Timestamp']=pd.to_datetime(data['Last QC Pending Timestamp'])

In [61]:
# type(data['Final TimeStamp'][0])

print(type(data['Last QC Pending Timestamp'][0]))
print(len(data))

<class 'pandas._libs.tslibs.nattype.NaTType'>
517290


In [62]:
data.columns

Index(['Merchant Cust ID', 'Lead_ID', 'Entity_Type', 'Node id', 'Lead_Stage',
       'Lead Status', 'Created Timestamp', 'Updated Timestamp', 'Ageing',
       'Product_Context', 'Name_Match_Status', '10K/1rs MID Status',
       'Final TimeStamp', 'QC Panel Successful TimeStamp',
       'Last QC Pending Timestamp', 'TAT', 'Merchant QC TAT slab',
       'Working TimeSlab', 'New Off Time', 'New Last QC Pending Time',
       'Stage movement count', 'Rejection Ageing', 'cust_id', 'agent_team',
       'mid', 'mid_created_date', 'solution_type_level_3'],
      dtype='object')

In [63]:
data['Last QC Pending Timestamp'].head(10)

0                   NaT
1                   NaT
2                   NaT
3   2025-04-10 19:41:49
4   2025-04-16 14:11:23
5   2025-04-09 17:56:23
6   2025-04-05 14:56:46
7   2025-04-19 11:39:54
8                   NaT
9   2025-04-11 11:28:54
Name: Last QC Pending Timestamp, dtype: datetime64[ns]

In [64]:


data['Last QC Pending Timestamp']=pd.to_datetime(data['Last QC Pending Timestamp'], errors='coerce')

# Define the start and end times
start_time = time(8, 30, 0)
end_time = time(21, 30, 0)



# Apply the condition to classify as 'Intime' or 'Offtime' based on the time component of 'N'
data['Working TimeSlab'] = data['Last QC Pending Timestamp'].apply(lambda x: 'Offtime' if pd.isna(x) else ('Intime' if start_time <= x.time() <= end_time else 'Offtime'))


data['Working TimeSlab'].head(10)

0    Offtime
1    Offtime
2    Offtime
3     Intime
4     Intime
5     Intime
6     Intime
7     Intime
8    Offtime
9     Intime
Name: Working TimeSlab, dtype: object

In [65]:
data['Working TimeSlab'].value_counts()

Intime     340467
Offtime    176823
Name: Working TimeSlab, dtype: int64

In [66]:
data.columns

Index(['Merchant Cust ID', 'Lead_ID', 'Entity_Type', 'Node id', 'Lead_Stage',
       'Lead Status', 'Created Timestamp', 'Updated Timestamp', 'Ageing',
       'Product_Context', 'Name_Match_Status', '10K/1rs MID Status',
       'Final TimeStamp', 'QC Panel Successful TimeStamp',
       'Last QC Pending Timestamp', 'TAT', 'Merchant QC TAT slab',
       'Working TimeSlab', 'New Off Time', 'New Last QC Pending Time',
       'Stage movement count', 'Rejection Ageing', 'cust_id', 'agent_team',
       'mid', 'mid_created_date', 'solution_type_level_3'],
      dtype='object')

In [67]:
out_time_start = pd.to_datetime("21:30").time()
out_time_end = pd.to_datetime("23:59:59").time()
morning_time_end = pd.to_datetime("08:30").time()

data['Last QC Pending Timestamp'] = pd.to_datetime(data['Last QC Pending Timestamp'], errors='coerce')


# Apply conditions to create column R
def process_timestamp(row):
    timestamp = row['Last QC Pending Timestamp']
    
    # Check if timestamp is NaT
    if pd.isna(timestamp):
        return np.nan
    
    time = timestamp.time()
    
    if row['Working TimeSlab'] == 'Offtime':
        if out_time_start <= time <= out_time_end:
            # Condition 1: Increment date by 1
            return timestamp + pd.DateOffset(days=1)
        elif time < morning_time_end:
            # Condition 2: Return the same timestamp
            return timestamp
    # Condition 3: Return blank for 'intime'
    return np.nan


data['New Off Time'] = data.apply(process_timestamp, axis=1)

In [68]:
# Function to create S column based on R
def generate_s(row):
    if pd.notna(row['New Off Time']):  # If R is not NaT or blank
        # Set time to 08:30 on the same date as R
        return row['New Off Time'].replace(hour=8, minute=30, second=0)
    else:
        # If R is NaT or blank, copy the value from Q
        return row['Last QC Pending Timestamp']
    

data['New Last QC Pending Time'] = data.apply(generate_s, axis=1)

In [69]:
data.columns

Index(['Merchant Cust ID', 'Lead_ID', 'Entity_Type', 'Node id', 'Lead_Stage',
       'Lead Status', 'Created Timestamp', 'Updated Timestamp', 'Ageing',
       'Product_Context', 'Name_Match_Status', '10K/1rs MID Status',
       'Final TimeStamp', 'QC Panel Successful TimeStamp',
       'Last QC Pending Timestamp', 'TAT', 'Merchant QC TAT slab',
       'Working TimeSlab', 'New Off Time', 'New Last QC Pending Time',
       'Stage movement count', 'Rejection Ageing', 'cust_id', 'agent_team',
       'mid', 'mid_created_date', 'solution_type_level_3'],
      dtype='object')

In [70]:
# Calculate the difference between L and S and format it as HH:mm:ss
def calculate_difference(row):
    if isinstance(row['New Last QC Pending Time'], pd.Timestamp) and isinstance(row['Final TimeStamp'], pd.Timestamp):
        diff = row['Final TimeStamp'] - row['New Last QC Pending Time']
        return diff  # Extract HH:mm:ss from Timedelta
    return np.nan  # Return NaN if either L or S is not a timestamp

# Apply function to create Difference column
data['TAT'] = data.apply(calculate_difference, axis=1)

In [71]:
data['TAT'].head(10)

0               NaT
1               NaT
2               NaT
3   0 days 00:02:51
4   0 days 00:05:51
5   0 days 00:01:27
6   0 days 00:03:35
7   0 days 00:17:34
8               NaT
9   0 days 00:05:53
Name: TAT, dtype: timedelta64[ns]

In [72]:
# Update TAT where it is negative to be the difference between L and N
def adjust_negative_tat(row):
    if pd.notna(row['TAT']) and row['TAT'] < pd.Timedelta(0):  # Check if TAT is negative
        if isinstance(row['Final TimeStamp'], pd.Timestamp) and isinstance(row['Last QC Pending Timestamp'], pd.Timestamp):
            return row['Final TimeStamp'] - row['Last QC Pending Timestamp']  # Calculate L - N if TAT is negative
    return row['TAT']  # Keep original TAT if not negative

# Apply function to adjust negative TAT values
data['TAT'] = data.apply(adjust_negative_tat, axis=1)

In [73]:
print(data[['Final TimeStamp','Last QC Pending Timestamp','TAT']].head(10))
print(len(data))

      Final TimeStamp Last QC Pending Timestamp             TAT
0 2025-04-02 15:04:42                       NaT             NaT
1 2025-04-07 14:11:43                       NaT             NaT
2 2025-04-26 12:00:13                       NaT             NaT
3 2025-04-10 19:44:40       2025-04-10 19:41:49 0 days 00:02:51
4 2025-04-16 14:17:14       2025-04-16 14:11:23 0 days 00:05:51
5 2025-04-09 17:57:50       2025-04-09 17:56:23 0 days 00:01:27
6 2025-04-05 15:00:21       2025-04-05 14:56:46 0 days 00:03:35
7 2025-04-19 11:57:28       2025-04-19 11:39:54 0 days 00:17:34
8 2025-04-25 18:49:36                       NaT             NaT
9 2025-04-11 11:34:47       2025-04-11 11:28:54 0 days 00:05:53
517290


In [74]:
len(data[((data['Lead Status']=='Merchant Live') | (data['Lead Status']=='Documents Rejected (Pending at Sales Team)'))&(data['TAT'].isnull())])

0

In [75]:
def categorize_time(td):
    total_minutes = td.total_seconds() / 60  # Convert timedelta to total minutes
    if total_minutes < 2:
        return 'Leads QCed in less than 02 min'
    elif 2 <= total_minutes < 5:
        return 'Leads QCed in between 02 - 05 min'
    elif 5 <= total_minutes < 10:
        return 'Leads QCed in between 05 - 10 min'
    elif 10 <= total_minutes < 15:
        return 'Leads QCed in between 10 - 15 min'
    elif 15 <= total_minutes < 30:
        return 'Leads QCed in between 15 - 30 min'
    elif 30 <= total_minutes < 60:
        return 'Leads QCed in between 30 min - 01 hr'
    else:
        return 'leads QCed in more than 01 hr'



data.loc[(data['Lead Status']=='Merchant Live') | (data['Lead Status']=='Documents Rejected (Pending at Sales Team)'),'Merchant QC TAT slab']=data['TAT'].apply(categorize_time)



In [76]:
data['Merchant QC TAT slab'].value_counts()

Leads QCed in between 02 - 05 min       148142
Leads QCed in less than 02 min          119085
Leads QCed in between 05 - 10 min        47376
Leads QCed in between 15 - 30 min        14997
Leads QCed in between 10 - 15 min        10171
Leads QCed in between 30 min - 01 hr      2538
leads QCed in more than 01 hr              841
Name: Merchant QC TAT slab, dtype: int64

In [77]:
len(data)

517290

In [78]:
len(data[data['solution_type_level_3']=='3P FSE'])

642

In [79]:
data['solution_type_level_3'].value_counts()

KYC_UPDATE         27853
ACCOUNT_UPGRADE    11899
3P FSE               642
DIY                   33
Name: solution_type_level_3, dtype: int64

In [80]:
data = data[~((data['Entity_Type'].isin(['PROPRIETORSHIP', 'NON_PROPRIETORSHIP'])) & (data['Product_Context'] == 'EDC'))]
len(data)

490292

In [81]:
data['solution_type_level_3'].value_counts()

KYC_UPDATE         27853
ACCOUNT_UPGRADE    11899
3P FSE               642
DIY                    1
Name: solution_type_level_3, dtype: int64

In [82]:
data.to_csv(out_file, index=False)

In [83]:
sas

NameError: name 'sas' is not defined

In [ ]:


# Convert to datetime
data['Last QC Pending Timestamp'] = pd.to_datetime(data['Last QC Pending Timestamp'])
data['Final TimeStamp'] = pd.to_datetime(data['Final TimeStamp'])

# Define active hours (08:30 AM - 09:30 PM)
active_start_time = pd.to_timedelta('08:30:00')
active_end_time = pd.to_timedelta('21:30:00')

# Function to calculate active duration for each row
def calculate_active_duration(row):
    start = row['Last QC Pending Timestamp']
    end = row['Final TimeStamp']
    
    # Check if either start or end is NaT
    if pd.isna(start) or pd.isna(end):
        return "00:00:00"  # Return 0 hours if either is NaT
    
    # Initialize total active duration
    total_active_duration = pd.Timedelta(0)
    
    # Iterate over each date from start to end
    for single_date in pd.date_range(start=start.date(), end=end.date()):
        active_start = pd.Timestamp(single_date) + active_start_time
        active_end = pd.Timestamp(single_date) + active_end_time
        
        # Determine the effective active period for this day
        effective_start = max(start, active_start)
        effective_end = min(end, active_end)

        # If there's a valid active period, add its duration
        if effective_start < effective_end:
            total_active_duration += effective_end - effective_start

    # Convert total active duration to hours
    total_hours = total_active_duration.total_seconds() / 3600
    
    # Format the hours to HH:MM:SS
    hours = int(total_hours)
    minutes = int((total_hours - hours) * 60)
    seconds = int(((total_hours - hours) * 60 - minutes) * 60)
    
    return f"{hours:02}:{minutes:02}:{seconds:02}"

# Apply the function to each row
data['TAT'] = data.apply(calculate_active_duration, axis=1)

# Display the result
print(data[['Last QC Pending Timestamp', 'Final TimeStamp', 'TAT']])
print(len(data))


In [ ]:
# data['TAT']=data['Final TimeStamp']-data['Last QC Pending Timestamp']
# data['TAT']=data['TAT'].apply(lambda x: str(x).split()[-1] if pd.notna(x) else 'NaT')
print(data.columns)
print(len(data))

In [ ]:
print(data[['Final TimeStamp','Last QC Pending Timestamp','TAT']].head(10))
print(len(data))

In [ ]:
len(data[((data['Lead Status']=='Merchant Live') | (data['Lead Status']=='Documents Rejected (Pending at Sales Team)'))&(data['TAT'].isnull())])

In [ ]:
data["TAT"]=pd.to_timedelta(data['TAT'])



print(data[['Final TimeStamp','Last QC Pending Timestamp','TAT']].head(10))

In [ ]:
type(data['TAT'][0])

In [ ]:
def categorize_time(td):
    total_minutes = td.total_seconds() / 60  # Convert timedelta to total minutes
    if total_minutes < 16:
        return 'Leads QCed in less than 15 min'
    elif 16 <= total_minutes < 36:
        return 'Leads QCed in between 15-30 min'
    elif 36 <= total_minutes < 66:
        return 'Leads QCed in between 30 min -1 hr'
    elif 66 <= total_minutes < 126:
        return 'Leads QCed in between 1 - 2 hr'
    elif 126 <= total_minutes < 186:
        return 'Leads QCed in between 2-3 hr'
    elif 186 <= total_minutes < 246:
        return 'Leads QCed in between 3-4 hr'
    else:
        return 'leads QCed in more than 4 hrs'



data.loc[(data['Lead Status']=='Merchant Live') | (data['Lead Status']=='Documents Rejected (Pending at Sales Team)'),'Merchant QC TAT slab']=data['TAT'].apply(categorize_time)


    



In [ ]:
print(data['Merchant QC TAT slab'].value_counts())
print(len(data))

In [ ]:
data['TAT'].head(10)

In [ ]:

# Function to convert timedelta to HH:MM:SS format with cumulative hours
def format_timedelta(td):
    total_seconds = int(td.total_seconds())
    hours, remainder = divmod(total_seconds, 3600)  # Convert total seconds to hours
    minutes, seconds = divmod(remainder, 60)
    return f'{hours:02}:{minutes:02}:{seconds:02}'  # Format as HH:MM:SS

# Apply the function to convert timedelta to HH:MM:SS
data['TAT'] = data['TAT'].apply(format_timedelta)

In [ ]:
data.loc[(data['Lead Status']!='Merchant Live') & (data['Lead Status']!='Documents Rejected (Pending at Sales Team)'),'TAT']=''

data['TAT'].head(10)

In [ ]:
data.columns

In [ ]:
data=data.drop(columns=['Working TimeSlab','New Off Time','New Last QC Pending Time','Node id'])
data.columns

In [ ]:
col=data.pop('TAT')

t=data.columns.get_loc('Merchant QC TAT slab')

data.insert(t+1,'TAT',col)

data.columns

In [ ]:
data['solution_type_level_3'].unique()
len(data)

In [ ]:
data_acup_kyc=data[data['solution_type_level_3'].isin(['ACCOUNT_UPGRADE', 'KYC_UPDATE'])]
data_mco=data[~data['solution_type_level_3'].isin(['ACCOUNT_UPGRADE', 'KYC_UPDATE'])]

In [ ]:
#raise Exception("Stop execution here")

In [ ]:
with pd.ExcelWriter("all_flow_data_21st Oct'24.xlsx") as writer:
    # Write the original dataframe to the first sheet
    data.to_excel(writer, sheet_name='All Data', index=False)
    
    # Write data filtered by Category 'A' and 'C' to the second sheet
    data_acup_kyc.to_excel(writer, sheet_name=' KYC & ACC UPGD', index=False)
    
    # Write data filtered by Category 'B' and 'D' to the third sheet
    data_mco.to_excel(writer, sheet_name='MCO', index=False)

In [ ]:
data.to_csv("all_flow_data_19th Oct'24.csv", index=False)
